### Figure 3B - Early vs Late Lysate Scatter Plot

Here we look at a read by read scatter plot of the early lysate sample vs the late lysate sample to see how much the cellular context has changed during the addition of stress and time

In [2]:
import pandas as pd
import numpy as np
import os
from bokeh.io import push_notebook, show, output_notebook, export_svg
output_notebook()
from bokeh.plotting import figure
import datetime

Loading BokehJS ...

First we will read our data frames into a file:

In [3]:
df_dir = '../../../Processed_Data_Files/dataframe_files/230322Li/'
df_dict = dict()
names = []
# Fetch all the relevant dataframe files and put them in a useable format:
for file in os.listdir(df_dir):
    if file.endswith('_dataframe.txt'):
        name = file[:]
        name = name[:name.find('_2')]
        new_df  = pd.read_csv(''.join([df_dir,file]))
        new_df['Length'] = new_df['Stop'] - new_df['Start']
        df_dict[name] = new_df
        names.append(name)
print(f'\nHere is the list of unique samples for which there are dataframe files: \n\n{names}\n')

# a name dictionary for converting between the datafram name and a form nicer for plots:
names_dict = {'Before-Ind': 'Before Induction', 'Early-Cap': "Early Capsule",
              'Early-Lys': "Early Lysate", 'Late-Cap': "Late Capsule",
              'Late-Lys': "Late Lysate", 'Mid-Stress': "Mid-Stress, Total Lysate",
              'No-Ind': "No Induction", 'Post-Ind': "Post Induction, Total Lysate"}


Here is the list of unique samples for which there are dataframe files: 

['Before-Ind', 'Early-Cap', 'Early-Lys', 'Late-Cap', 'Late-Lys', 'Mid-Stress', 'No-Ind', 'Post-Ind']



Some helper functions to help us with the plotting:

In [4]:
def into_RPM(df_dict, name):
    '''This felper function gets the dataframe for a particular experiment and converts its reads into RPM'''
    sample_df = df_dict[name].copy()
    sample_df = sample_df.rename(columns = {'Counts':name})
    sample_df[name] = sample_df[name]/(sum(sample_df[name])/1000000) #RPM]
    
    return sample_df

Now we will plot a simple scatter plot comparing the early and late Lysate samples on a read by read basis:

In [5]:
x = 'Early-Lys' # Early whole cell lysate reads
y = 'Late-Lys' # Late whole cell lysate reads

x_df = into_RPM(df_dict, x)
y_df = into_RPM(df_dict, y)
plot_df = x_df.merge(y_df)
plot_df.drop(plot_df[plot_df['Name'] == 'error'].index, inplace = True) # currently dropping any gene without reads in both
plot_df.drop(plot_df[plot_df[x] == 0].index, inplace = True)
plot_df.drop(plot_df[plot_df[y] == 0].index, inplace = True)
plot_df['Log_Dif'] = np.log10(plot_df[x]) - np.log10(plot_df[y])

p = figure(
    y_axis_type = "log", x_axis_type = "log",
    aspect_scale = 1, width = 385, height = 385,
    output_backend = "svg", tooltips = [("Gene", "@Name")]
)

#configure title appearance:
p.title = "Lysate RNA Levels Pre and Post Stress"
p.title.text_font = "arial"
p.title.text_font_size = "20px"
p.title.align = "center"
p.title.text_font_style = 'normal'

p.xaxis.axis_label = "Pre-Stress RNA Levels (RPM)"
p.yaxis.axis_label = "Post-Stress RNA Levels (RPM)"
p.axis.axis_label_text_font_style = 'normal'
p.axis.ticker = [10**0, 10**2, 10**4, 10**6]
p.axis.major_label_text_font = "arial"
p.axis.major_label_text_font_size = "20px"
p.axis.axis_label_text_font = "arial"
p.axis.axis_label_text_font_size = "20px"

p.background_fill_color = "whitesmoke"
p.grid.grid_line_color = "white"
p.grid.grid_line_width = 4

within_order_mag = plot_df[np.abs(plot_df['Log_Dif']) < 1]
greater_than_order_mag = plot_df[np.abs(plot_df['Log_Dif']) >= 1]

p.circle(x=x, y=y, source=within_order_mag, size=7, fill_alpha=0.8, line_alpha=0, color="darkslateblue")
p.circle(x=x, y=y, source=greater_than_order_mag, size=7, fill_alpha=0.8, line_alpha=0, color="orangered")
p.line(x=[10**(-1), 10**5], y=[10**(-1), 10**5], color = "darkseagreen", width = 3.5)
p.line(x=[10*10**(-1), 10**5], y=[10**(-1), 10**4], color = "darkseagreen", width = 4)
p.line(x=[10**(-1), 10**4], y=[10**(0), 10**5], color = "darkseagreen", width = 4)

show(p)
export_svg(p, filename = f'./3B_{datetime.date.today()}.svg')

['./3B_2023-08-02.svg']

In [5]:
print(len(greater_than_order_mag))

175
